In [ ]:
%%writefile app.py 

# Para este trabajo se utlizarón mis ideas, los códigos vistos en clase y consultas de IA

#Creamos el archivo de la APP en el interprete principal (Phyton)
#####################################################
#Importamos librerias
import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
######################################################
#Definimos la instancia
@st.cache_resource
######################################################
#Creamos la función de carga de datos
def load_data():
   #Lectura del archivo csv
   df=pd.read_csv("Mexico.csv")
   Lista=[
   # Identificación y ubicación
    'id', 'host_location', 'neighbourhood_cleansed', 'latitude', 'longitude',

    # Características del alojamiento
    'property_type', 'room_type', 'accommodates', 'accommodates_cat', 'accommodates_bin',
    'bathrooms', 'bathrooms_bin', 'bathrooms_cat',
    'bedrooms', 'bedrooms_bin', 'bedrooms_cat',
    'beds', 'amenities',

    # Precio y disponibilidad
    'price', 'minimum_nights', 'maximum_nights',
    'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm',
    'min_nights_bin', 'min_nights_cat',
    'has_availability', 'availability_30', 'availability_60',
    'availability_90', 'availability_365',
    'estimated_occupancy_l365d', 'estimated_revenue_l365d',

    # Evaluaciones
    'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
    'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
    'review_scores_checkin', 'review_scores_communication',
    'review_scores_location', 'review_scores_value',
    'review_scores_val_bin', 'review_scores_val_cat',

    # Información del host
    'host_is_superhost', 'host_is_superhost_bin',
    'host_has_profile_pic', 'host_has_profile_pic_bin',
    'host_identity_verified', 'host_identity_verified_bin',
    'instant_bookable', 'instant_bookable_bin',
    'source_bin'
]
   return df, Lista

###############################################################################
#Cargo los datos obtenidos de la función "load_data"
df, Lista = load_data()
###############################################################################
#CREACIÓN DEL DASHBOARD
#Generamos las páginas que utilizaremos en el diseño
##############################################################################
#Generamos los encabezados para la barra lateral (sidebar)
st.sidebar.title("Ciudad de México - Análisis de Datos de Airbnb")
#Widget 1: Selectbox
#Menu desplegable de opciones de laa páginas seleccionadas
View = st.sidebar.selectbox(
    label="Selecciona el tipo de análisis",
    options=["Vista México",
        "Exploración de variables",
        "Regresión Lineal",
        "Regresión No Lineal",
        "Regresión Logística"
    ] )
##############################################################################
# VISTA MÉXICO: Análisis general de alojamientos en México
if View == "Vista México":
    # Título principal del dashboard
    st.title("Análisis de Alojamientos en Ciudad de México")

    # BLOQUE 1: Histograma de precios
    st.subheader("Distribución de precios")
    fig_precio = px.histogram(
        df,
        x='price',
        nbins=50,
        title='Distribución de precios',
        color_discrete_sequence=['#FF5A5F']
    )
    fig_precio.update_layout(title_font_color='#FF5A5F')
    st.plotly_chart(fig_precio, use_container_width=True)

    # BLOQUE 2: Mapa interactivo
    st.subheader("Mapa de alojamientos")
    fig_mapa = px.scatter_mapbox(
        df,
        lat="latitude",
        lon="longitude",
        color="price",
        size="review_scores_value",
        hover_name="property_type",
        hover_data=["room_type", "accommodates", "beds"],
        color_continuous_scale=['#FFE1E3', '#FF5A5F', '#C1353A'],
        zoom=10,
        height=500
    )
    fig_mapa.update_layout(mapbox_style="carto-positron")
    fig_mapa.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    st.plotly_chart(fig_mapa, use_container_width=True)

    # BLOQUE 3: Evaluación promedio por tipo de alojamiento
    st.subheader("Evaluación promedio por tipo de alojamiento")
    promedio_eval = df.groupby('room_type')['review_scores_value'].mean().reset_index()
    fig_eval = px.bar(
        promedio_eval,
        x='room_type',
        y='review_scores_value',
        title='Evaluación promedio por tipo de habitación',
        color_discrete_sequence=['#FF5A5F']
    )
    fig_eval.update_layout(title_font_color='#FF5A5F')
    st.plotly_chart(fig_eval, use_container_width=True)

    # BLOQUE 4: Disponibilidad promedio por tipo de alojamiento
    st.subheader("Disponibilidad promedio por tipo de alojamiento")
    disponibilidad = df.groupby('room_type')['availability_365'].mean().reset_index()
    fig_disp = px.bar(
        disponibilidad,
        x='room_type',
        y='availability_365',
        title='Días disponibles al año por tipo de habitación',
        color_discrete_sequence=['#FF5A5F']
    )
    fig_disp.update_layout(title_font_color='#FF5A5F')
    st.plotly_chart(fig_disp, use_container_width=True)

    # BLOQUE 5: Porcentaje de superhosts por tipo de alojamiento
    st.subheader("Porcentaje de superhosts por tipo de alojamiento")
    superhost_ratio = df.groupby('room_type')['host_is_superhost_bin'].mean().reset_index()
    fig_superhost = px.bar(
        superhost_ratio,
        x='room_type',
        y='host_is_superhost_bin',
        title='Proporción de superhosts por tipo de habitación',
        color_discrete_sequence=['#FF5A5F']
    )
    fig_superhost.update_layout(title_font_color='#FF5A5F')
    st.plotly_chart(fig_superhost, use_container_width=True)

    # BLOQUE 6: Capacidad promedio por tipo de alojamiento
    st.subheader("Capacidad promedio por tipo de alojamiento")
    capacidad = df.groupby('room_type')['accommodates'].mean().reset_index()
    fig_capacidad = px.bar(
        capacidad,
        x='room_type',
        y='accommodates',
        title='Capacidad promedio por tipo de habitación',
        color_discrete_sequence=['#FF5A5F']
    )
    fig_capacidad.update_layout(title_font_color='#FF5A5F')
    st.plotly_chart(fig_capacidad, use_container_width=True)

##############################################################################
# VISTA EXPLORACIÓN DE VARIABLES: Análisis exploratorio de variables
if View == "Exploración de variables":
    st.title("Exploración de Variables - Alojamientos en México")

    df_cat = df.select_dtypes(include='object')
    df_num = df.select_dtypes(include=['float', 'int'])

    tipo_variable = st.sidebar.radio("Tipo de variable", ["Categórica", "Numérica"])

    if tipo_variable == "Categórica":
        variable_cat = st.sidebar.selectbox("Variable categórica", options=df_cat.columns)
        tipo_grafico = st.sidebar.radio("Tipo de gráfico", ["Barras", "Pastel", "Dona"])

        tabla_frec = df[variable_cat].value_counts().nlargest(10).reset_index()
        tabla_frec.columns = ['categoría', 'frecuencia']

        st.subheader(f"Top 10 categorías de '{variable_cat}'")

        if tipo_grafico == "Barras":
            fig = px.bar(tabla_frec, x='categoría', y='frecuencia',
                         title='Frecuencia por categoría',
                         color_discrete_sequence=['#FF5A5F'])
        elif tipo_grafico == "Pastel":
            fig = px.pie(tabla_frec, names='categoría', values='frecuencia',
                         title='Distribución en gráfico de pastel',
                         color_discrete_sequence=['#FF5A5F'])
        else:  # Dona
            fig = px.pie(tabla_frec, names='categoría', values='frecuencia',
                         hole=0.4,
                         title='Distribución en gráfico de dona',
                         color_discrete_sequence=['#FF5A5F'])

        fig.update_layout(title_font_color='#FF5A5F')
        st.plotly_chart(fig, use_container_width=True)

    else:
        variable_num = st.sidebar.selectbox("Variable numérica", options=df_num.columns)
        tipo_grafico = st.sidebar.radio("Tipo de gráfico", ["Histograma", "Boxplot"])

        st.subheader(f"Distribución de '{variable_num}'")

        df_filtrado = df[[variable_num]].dropna()

        if tipo_grafico == "Histograma":
            fig = px.histogram(df_filtrado, x=variable_num, nbins=50,
                               title='Histograma de distribución',
                               color_discrete_sequence=['#FF5A5F'])
        else:  # Boxplot
            fig = px.box(df_filtrado, y=variable_num,
                         title='Boxplot de distribución',
                         color_discrete_sequence=['#FF5A5F'])

        fig.update_layout(title_font_color='#FF5A5F')
        st.plotly_chart(fig, use_container_width=True)



##############################################################################
# VISTA REGRESIÓN LINEAL: Análisis de regresión lineal
if View == "Regresión Lineal":
    st.title("Regresión Lineal - Alojamientos en Ciudad de México")

    # Lista de variables numéricas disponibles
    numeric_df = df.select_dtypes(include=['float', 'int'])
    Lista_num = numeric_df.columns.tolist()

    # Selección de variables para regresión simple
    Variable_y = st.sidebar.selectbox("Variable objetivo (Y)", options=Lista_num)
    Variable_x = st.sidebar.selectbox("Variable independiente (X)", options=Lista_num)

    # Fila 1: Regresión Lineal Simple
    Contenedor_A, Contenedor_B = st.columns(2)
    with Contenedor_A:
        st.subheader("Regresión Lineal Simple")

        # Modelo lineal simple
        from sklearn.linear_model import LinearRegression
        modelo_simple = LinearRegression()
        modelo_simple.fit(df[[Variable_x]], df[Variable_y])
        y_pred_simple = modelo_simple.predict(df[[Variable_x]])

        # Métricas
        r2_simple = modelo_simple.score(df[[Variable_x]], df[Variable_y])
        correl_simple = np.sqrt(r2_simple)
        st.write(f"Coeficiente de correlación: {correl_simple:.4f}")

        # Gráfico de dispersión con línea de regresión
        fig_simple = px.scatter(
            df,
            x=Variable_x,
            y=Variable_y,
            title="Modelo Lineal Simple",
            color_discrete_sequence=['#FF5A5F']
        )
        fig_simple.add_traces(
            px.line(
                x=df[Variable_x],
                y=y_pred_simple,
                color_discrete_sequence=['#FF5A5F']
            ).data
        )
        fig_simple.update_layout(title_font_color='#FF5A5F')
        st.plotly_chart(fig_simple, use_container_width=True)

    # Fila 2: Regresión Lineal Múltiple
    with Contenedor_B:
        st.subheader("Regresión Lineal Múltiple")

        Variables_x = st.sidebar.multiselect(
            "Variables independientes (X)",
            options=Lista_num,
            default=[Variable_x]
        )

        if Variables_x:
            modelo_multiple = LinearRegression()
            modelo_multiple.fit(df[Variables_x], df[Variable_y])
            y_pred_multiple = modelo_multiple.predict(df[Variables_x])

            r2_multiple = modelo_multiple.score(df[Variables_x], df[Variable_y])
            correl_multiple = np.sqrt(r2_multiple)
            st.write(f"Coeficiente de correlación múltiple: {correl_multiple:.4f}")

            # Gráfico de dispersión con variable más correlacionada
            var_principal = Variables_x[0]
            fig_multiple = px.scatter(
                df,
                x=var_principal,
                y=Variable_y,
                title=f"Modelo Lineal Múltiple (vista sobre {var_principal})",
                color_discrete_sequence=['#FF5A5F']
            )
            fig_multiple.update_layout(title_font_color='#FF5A5F')
            st.plotly_chart(fig_multiple, use_container_width=True)
        else:
            st.warning("Selecciona al menos una variable independiente para el modelo múltiple.")

##############################################################################
# VISTA REGRESIÓN NO LINEAL: Análisis de regresión no lineal
if View == "Regresión No Lineal":
    st.title("Regresión No Lineal - Alojamientos en Ciudad de México")

    # Lista de variables numéricas
    numeric_df = df.select_dtypes(include=['float', 'int'])
    Lista_num = numeric_df.columns.tolist()

    # Selección de variables
    Variable_y = st.sidebar.selectbox("Variable objetivo (Y)", options=Lista_num)
    Variable_x = st.sidebar.selectbox("Variable independiente (X)", options=Lista_num)

    # Selección de modelo
    Lista_mod = ["Función cuadrática", "Función exponencial"]
    Modelo = st.sidebar.selectbox("Modelo no lineal", options=Lista_mod)

    # Extraer variables
    x = df[Variable_x].dropna()
    y = df[Variable_y].dropna()
    df_modelo = df[[Variable_x, Variable_y]].dropna()

    # Ajuste del modelo
    from scipy.optimize import curve_fit
    from sklearn.metrics import r2_score

    if Modelo == "Función cuadrática":
        def func(x, a, b, c):
            return a * x**2 + b * x + c
    elif Modelo == "Función exponencial":
        def func(x, a, b, c):
            return a * np.exp(-b * x) + c

    try:
        parametros, _ = curve_fit(func, x, y)
        y_pred = func(x, *parametros)
        r2 = r2_score(y, y_pred)
        correl = np.sqrt(r2)

        st.subheader("Resultados del modelo")
        st.write(f"Coeficiente de correlación: {correl:.4f}")
        st.write(f"Coeficiente de determinación (R²): {r2:.4f}")

        # Gráfico con curva ajustada
        fig = px.scatter(
            df_modelo,
            x=Variable_x,
            y=Variable_y,
            title=f"Regresión No Lineal - {Modelo}",
            color_discrete_sequence=['#FF5A5F']
        )
        fig.add_traces(
            px.line(
                x=x.sort_values(),
                y=func(x.sort_values(), *parametros),
                color_discrete_sequence=['#FF5A5F']
            ).data
        )
        fig.update_layout(title_font_color='#FF5A5F')
        st.plotly_chart(fig, use_container_width=True)

    except Exception as e:
        st.error("No se pudo ajustar el modelo. Verifica que las variables seleccionadas tengan suficiente variación.")
        st.write(f"Error: {e}")

##############################################################################
# VISTA REGRESIÓN LOGÍSTICA: Análisis de regresión logística
if View == "Regresión Logística":
    st.title("Regresión Logística - Alojamientos en Ciudad de México")

    # Lista de variables numéricas y dicotómicas
    numeric_df = df.select_dtypes(include=['float', 'int'])
    Lista_num = numeric_df.columns.tolist()

    # Variables dicotómicas 
    Lista_dicot = [
        'host_is_superhost_bin',
        'instant_bookable_bin',
        'host_identity_verified_bin',
        'host_has_profile_pic_bin',
        'review_scores_val_bin',
        'accommodates_bin',
        'bathrooms_bin',
        'bedrooms_bin',
        'min_nights_bin'
    ]

    # Selección de variables
    Variable_y = st.sidebar.selectbox("Variable dependiente (Y)", options=Lista_dicot)
    Variables_x = st.sidebar.multiselect(
        "Variables independientes (X)",
        options=Lista_num,
        default=['price', 'accommodates']
    )

    # Validación
    if not Variables_x:
        st.warning("Selecciona al menos una variable independiente.")
    else:
        # Preparación de datos
        X = df[Variables_x].dropna()
        y = df[Variable_y].loc[X.index]

        # División de datos
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

        # Escalamiento
        from sklearn.preprocessing import StandardScaler
        escalar = StandardScaler()
        X_train = escalar.fit_transform(X_train)
        X_test = escalar.transform(X_test)

        # Modelo
        from sklearn.linear_model import LogisticRegression
        modelo_log = LogisticRegression()
        modelo_log.fit(X_train, y_train)
        y_pred = modelo_log.predict(X_test)

        # Métricas
        from sklearn.metrics import confusion_matrix, accuracy_score, precision_score
        matriz = confusion_matrix(y_test, y_pred)
        exactitud = accuracy_score(y_test, y_pred)
        clases = np.unique(y)
        labels = [clases[0], clases[1]]

        # Heatmap de matriz de confusión
        import plotly.graph_objects as go
        figure_log = go.Figure(data=go.Heatmap(
            z=matriz,
            x=labels,
            y=labels,
            hoverinfo="z",
            colorscale=[[0, '#FFE1E3'], [1, '#FF5A5F']],
            showscale=True,
            zmin=0
        ))

        # Anotaciones TP, FP, FN, TN
        annotations = []
        for i in range(matriz.shape[0]):
            for j in range(matriz.shape[1]):
                valor = matriz[i, j]
                texto = f"{['TP','FP','FN','TN'][i*2+j]}: {valor}"
                annotations.append(
                    dict(
                        x=labels[j],
                        y=labels[i],
                        text=texto,
                        showarrow=False,
                        font=dict(color="white" if valor > matriz.max()/2 else "black")
                    )
                )

        figure_log.update_layout(
            title='Matriz de Confusión',
            xaxis_title="Predicción",
            yaxis_title="Real",
            annotations=annotations,
            width=500,
            height=500,
            title_font_color='#FF5A5F'
        )
        st.plotly_chart(figure_log)

        # Métricas adicionales
        st.subheader("Métricas del modelo")
        st.write(f"Exactitud: {exactitud:.4f}")
        st.write(f"Precisión para {clases[0]}: {precision_score(y_test, y_pred, pos_label=clases[0]):.4f}")
        st.write(f"Precisión para {clases[1]}: {precision_score(y_test, y_pred, pos_label=clases[1]):.4f}")



Overwriting app.py
